In [1]:
import pandas as pd

from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options

import concurrent.futures 

from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from bs4 import BeautifulSoup as bs
import requests as req

import time

In [2]:
#Cookies 
options=Options()
options.add_argument(r'user-data-dir=C:\Users\matia\anaconda3\Lib\site-packages\selenium\cookies') #Cargamos las cookies

In [3]:
url = 'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query=abner'

PATH = 'driver/chromedriver.exe'

In [4]:
driver = webdriver.Chrome(PATH, options=options)

driver.get(url)

In [5]:
#Scrapeo para sacar el valor de un jugador
driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text

'7,00 mill. €'

In [6]:
#Hago una funcion para scrapear el valor de todos mis jugadores
def valor(futbolista,PATH):
    
    driver = webdriver.Chrome(PATH) 
    time.sleep(2)
    driver.get(f'https://www.transfermarkt.es/schnellsuche/ergebnis/schnellsuche?query={futbolista}')
    time.sleep(2)
    try:
        valor = driver.find_elements(By.CSS_SELECTOR, '#yw0 > table > tbody > tr:nth-child(1) > td.rechts.hauptlink')[0].text
    except:
        valor = 'NaN'
        print(f'No se encontro el valor de {futbolista}')

    driver.quit()
    return valor


In [7]:
#Pruebo la funcion
valor('Carvajal',PATH)

'12,00 mill. €'

# Lo aplico a mi dataframe


In [8]:
df = pd.read_csv('../../csv/csv_limpios/jugadores_2022_lp.csv')

In [9]:
lista_jugadores_22 = df['Jugador'].tolist()

In [10]:
with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores_22]
    resultados = [f.result() for f in concurrent.futures.as_completed(futures)]

In [14]:
df['Valor Mercado'] = resultados

# Lo aplico a mi dataframe de 2023


In [26]:
df2 = pd.read_csv('../../csv/csv_limpios/jugadores_2023_lp.csv')

In [28]:
lista_jugadores_23 = df2['Jugador'].tolist()
len(lista_jugadores_23)

479

In [21]:
with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:  # Limita el número de hilos  
    futures = [executor.submit(valor, jugador, PATH) for jugador in lista_jugadores_23]
    resultados_23 = [f.result() for f in concurrent.futures.as_completed(futures)]

In [23]:
df2['Valor Mercado'] = resultados_23

ValueError: Length of values (566) does not match length of index (479)